In [7]:
import pandas as pd
samsung_df = pd.read_csv('SSNG.csv', delimiter=';')
apple_df = pd.read_csv('AAPL.csv', delimiter=',')

print(samsung_df.head())
print(apple_df.head())

         Date        Open        High         Low       Close   Adj Close  \
0  04/01/2000  6000000000  6110000000  5660000000  6110000000  4651737793   
1  05/01/2000  5800000000  6060000000  5520000000  5580000000  4248232422   
2  06/01/2000  5750000000  5780000000  5580000000  5620000000  4278686523   
3  07/01/2000  5560000000  5670000000  5360000000  5540000000  4217780273   
4  10/01/2000  5600000000  5770000000  5580000000  5770000000  4392884766   

     Volume  
0  74195000  
1  74680000  
2  54390000  
3  40305000  
4  46880000  
         Date      Open      High       Low     Close  Adj Close     Volume
0  1980-12-12  0.128348  0.128906  0.128348  0.128348   0.099319  469033600
1  1980-12-15  0.122210  0.122210  0.121652  0.121652   0.094137  175884800
2  1980-12-16  0.113281  0.113281  0.112723  0.112723   0.087228  105728000
3  1980-12-17  0.115513  0.116071  0.115513  0.115513   0.089387   86441600
4  1980-12-18  0.118862  0.119420  0.118862  0.118862   0.091978   734496